In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 125
import seaborn as sns
from Bio import SeqIO, Seq
import scipy.stats as st
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import glob, os, yaml, subprocess, itertools, sparse, pickle

who_variants = pd.read_csv("/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv")
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Combined Analysis Files

In [12]:
drug = "Moxifloxacin"
mxf_analysis = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")

drug = "Levofloxacin"
lev_analysis = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")

drug = "Amikacin"
ami_analysis = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")

In [14]:
ami_analysis

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,OR_LB,OR_UB,Tier1_only,WHO_phenos,poolLOF,Syn
0,rrs_n.1401A>G,1.4478,1.4008,1.5091,0.0000,0.0000,0.0000,1473246.0,1) Assoc w R,4.2538,4.0584,4.5229,1,1,1,0
1,eis_c.-14C>T,0.3314,0.2683,0.3948,0.0000,0.0000,0.0000,2715346.0,1) Assoc w R,1.3930,1.3078,1.4841,1,1,1,0
2,rrs_n.1484G>T,0.1729,0.1367,0.2138,0.0000,0.0000,0.0000,1473329.0,2) Assoc w R - Interim,1.1888,1.1465,1.2384,1,1,1,0
3,rrs_n.514A>C,0.1209,0.0460,0.1927,0.0007,0.0083,0.4168,1472359.0,5) Not assoc w R,1.1285,1.0471,1.2125,1,1,1,0
4,rrs_n.1402C>T,0.1040,0.0383,0.1514,0.0002,0.0030,0.1179,1473247.0,2) Assoc w R - Interim,1.1096,1.0391,1.1634,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,eis_p.Asp325fs,-0.0123,-0.0205,0.0000,0.0411,0.0653,1.0000,NaN,NaN,0.9878,0.9797,1.0000,1,0,0,0
1973,whiB7_p.Arg12fs,-0.0158,-0.0220,0.0000,0.0320,0.0818,1.0000,NaN,NaN,0.9843,0.9783,1.0000,1,0,0,0
1974,eis_p.Leu191fs,-0.0196,-0.0260,0.0000,0.0303,0.0979,1.0000,NaN,NaN,0.9806,0.9743,1.0000,1,0,0,0
1975,eis_p.Val85fs,-0.0443,-0.0524,0.0000,0.0033,0.0240,1.0000,NaN,NaN,0.9567,0.9490,1.0000,1,0,0,0


In [10]:
mxf_analysis.query("Tier1_only==0 & WHO_phenos==0 & poolLOF==0")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,OR_LB,OR_UB,Tier1_only,WHO_phenos,poolLOF,Syn
2524,Rv2752c_p.Val348fs,0.0781,0.0308,0.1130,0.0001,0.0014,0.1799,NaN,NaN,1.0812,1.0313,1.1197,0,0,0,0
2525,glpK_p.Val192fs,0.0707,0.0087,0.1286,0.0100,0.0427,1.0000,NaN,NaN,1.0733,1.0088,1.1373,0,0,0,0
2526,glpK_p.Thr455fs,0.0493,0.0000,0.0584,0.0249,0.0889,1.0000,NaN,NaN,1.0506,1.0000,1.0601,0,0,0,0
2527,glpK_p.His193fs,0.0464,-0.0272,0.0856,0.0452,0.0593,1.0000,NaN,NaN,1.0475,0.9732,1.0894,0,0,0,0
2528,Rv2752c_p.Ser339*,0.0440,0.0000,0.0510,0.0253,0.0888,1.0000,NaN,NaN,1.0450,1.0000,1.0523,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,Rv2752c_p.Gln247*,-0.0540,-0.0985,0.0013,0.0167,0.0649,1.0000,NaN,NaN,0.9474,0.9062,1.0013,0,0,0,0
2637,Rv2752c_p.Val388fs,-0.0590,-0.0669,0.0000,0.0031,0.0241,1.0000,NaN,NaN,0.9427,0.9353,1.0000,0,0,0,0
2638,Rv2752c_p.Arg97fs,-0.0620,-0.0943,-0.0239,0.0002,0.0020,0.2849,NaN,NaN,0.9399,0.9100,0.9764,0,0,0,0
2639,Rv2752c_p.Gly56fs,-0.0664,-0.0851,-0.0287,0.0001,0.0011,0.1317,NaN,NaN,0.9358,0.9184,0.9717,0,0,0,0


In [11]:
mxf_analysis.query("Tier1_only==0 & WHO_phenos==0 & Syn==1")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,OR_LB,OR_UB,Tier1_only,WHO_phenos,poolLOF,Syn
370,Rv2752c_c.675C>T,0.1101,0.0470,0.1568,0.0000,0.0009,0.1172,NaN,NaN,1.1164,1.0481,1.1698,0,0,1,1
371,glpK_lof,0.1000,0.0384,0.1619,0.0008,0.0085,1.0000,NaN,NaN,1.1052,1.0392,1.1758,0,0,1,1
372,gyrA_c.2022C>G,0.0618,0.0000,0.0726,0.0063,0.0339,1.0000,NaN,NaN,1.0637,1.0000,1.0753,0,0,1,1
373,gyrA_p.Arg392Cys,0.0615,0.0000,0.0737,0.0037,0.0278,1.0000,NaN,NaN,1.0634,1.0000,1.0764,0,0,1,1
374,glpK_c.117G>C,0.0603,-0.0284,0.0948,0.0310,0.0954,1.0000,NaN,NaN,1.0621,0.9720,1.0995,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,glpK_p.Leu228Val,-0.0906,-0.1457,-0.0385,0.0018,0.0170,1.0000,NaN,NaN,0.9134,0.8644,0.9622,0,0,1,1
2520,Rv2477c_c.1545C>T,-0.1315,-0.2418,-0.0283,0.0088,0.0421,1.0000,NaN,NaN,0.8767,0.7852,0.9721,0,0,1,1
2521,Rv2477c_p.Gly41Glu,-0.1405,-0.2298,-0.0660,0.0005,0.0057,1.0000,NaN,NaN,0.8689,0.7947,0.9361,0,0,1,1
2522,glpK_c.1134C>T,-0.1410,-0.2736,-0.0166,0.0185,0.0749,1.0000,NaN,NaN,0.8685,0.7606,0.9836,0,0,1,1


In [4]:
mxf_analysis.Tier1_only.value_counts()

0    2271
1     406
Name: Tier1_only, dtype: int64

In [5]:
mxf_analysis.WHO_phenos.value_counts()

0    2307
1     370
Name: WHO_phenos, dtype: int64

In [6]:
mxf_analysis.poolLOF.value_counts()

1    2524
0     153
Name: poolLOF, dtype: int64

In [7]:
mxf_analysis.Syn.value_counts()

1    2176
0     501
Name: Syn, dtype: int64

# Catalog-Based Method Comparison

In [20]:
def get_logReg_summary(out_dir):
    
    model_summary = pd.read_csv(os.path.join(out_dir, "logReg_summary.csv"))
    model_analysis = pd.read_csv(os.path.join(out_dir, "model_analysis.csv"))
    
    summary_sens = model_summary.loc[0, "Sens"]
    summary_spec = model_summary.loc[0, "Spec"]
    
    #max_idx = [np.argmax(model_analysis["Sens"]), np.argmax(model_analysis["Spec"])]
    
    better_variant = model_analysis.query("Sens >= @summary_sens & Spec >= @summary_spec")
    
    if len(better_variant) == 0:
        print("No variants have comparable sensitivity AND specificity")
    else:
        print(better_variant)
    return model_summary, model_analysis
    
    #return model_summary, model_analysis.iloc[max_idx, :][["orig_variant", "coef", "Sens", "Spec", "accuracy", "balanced_accuracy"]]

In [26]:
#out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Levofloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn"

summary, analysis = get_logReg_summary(out_dir)
summary

No variants have comparable sensitivity AND specificity


,Sens,Spec,accuracy,balanced_accuracy
0,0.8336,0.9084,0.8951,0.8710


In [27]:
analysis.query("Sens > 0.9006")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,accuracy,balanced_accuracy


In [29]:
analysis.loc[analysis["orig_variant"].str.contains("PC")]

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,accuracy,balanced_accuracy
35,PC2,0.1660,0.0355,0.3034,0.0078,0.0301,1.0000,NaN,NaN,1.1806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,PC0,0.1346,0.0166,0.2630,0.0140,0.0492,1.0000,NaN,NaN,1.1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
analysis.query("Spec > 0.9575")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,accuracy,balanced_accuracy
0,pncA_c.-11A>G,0.4088,0.3700,0.4467,0.0000,0.0000,0.0000,2289252.0000,1) Assoc w R,1.5050,...,0.0274,0.0395,0.9993,0.9999,44.4364,284.1349,0.9608,0.9731,0.8278,0.5163
1,pncA_p.His51Asp,0.3308,0.3054,0.3591,0.0000,0.0000,0.0000,2289091.0000,1) Assoc w R,1.3921,...,0.0095,0.0177,0.9998,1.0000,63.1368,inf,0.9823,0.9905,0.8246,0.5067
2,pncA_p.Val131fs,0.2900,0.2684,0.3154,0.0000,0.0000,0.0000,NaN,NaN,1.3365,...,0.0090,0.0165,0.9998,1.0000,54.4333,inf,0.9836,0.9912,0.8245,0.5064
3,pncA_p.Gln141Pro,0.2869,0.2529,0.3273,0.0000,0.0000,0.0000,2288820.0000,1) Assoc w R,1.3323,...,0.0151,0.0243,0.9995,0.9999,35.3919,344.8754,0.9759,0.9851,0.8256,0.5097
4,pncA_p.Leu172Pro,0.2779,0.2524,0.3046,0.0000,0.0000,0.0000,2288727.0000,1) Assoc w R,1.3204,...,0.0062,0.0129,1.0000,1.0000,inf,inf,0.9871,0.9938,0.8239,0.5046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,rpsA_p.Ile70Leu,-0.1249,-0.1356,-0.1120,0.0000,0.0000,0.0000,NaN,NaN,0.8826,...,0.0000,0.0000,0.9966,0.9982,0.0000,0.0000,1.0018,1.0034,0.1798,0.5013
855,PPE35_p.Pro670Leu,-0.1287,-0.1817,-0.0860,0.0000,0.0000,0.0009,2168604.0000,5) Not assoc w R,0.8792,...,0.0003,0.0024,0.9800,0.9842,0.0156,0.1377,1.0145,1.0196,0.1922,0.5084
856,Rv1258c_p.Gly363Val,-0.1288,-0.1399,-0.1158,0.0000,0.0000,0.0000,1406253.0000,3) Uncertain significance,0.8791,...,0.0000,0.0000,0.9965,0.9981,0.0000,0.0000,1.0019,1.0036,0.1799,0.5013
857,Rv3236c_c.546C>T,-0.1577,-0.1859,-0.1337,0.0000,0.0000,0.0000,NaN,NaN,0.8541,...,0.0000,0.0000,0.9916,0.9942,0.0000,0.0000,1.0058,1.0085,0.1835,0.5035


In [19]:
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Moxifloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"

get_logReg_summary(out_dir)

No variants have comparable sensitivity AND specificity


,Sens,Spec,accuracy,balanced_accuracy
0,0.9230,0.8945,0.8983,0.9088


In [38]:
df_pza = pd.read_csv("/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn/phenos.csv")

# the missing ones might be M. cannettii, most similar to L6 based on the other lineage callers
lineages = pd.read_pickle("../data/combined_lineage_sample_IDs.pkl")
lineages["Lineage"] = lineages["Lineage"].fillna("6")
lineages["Lineage_1"] = lineages["Lineage_1"].fillna("6")

lineages = lineages[["Sample Name", "Sample ID", "Lineage_1"]]
lineages["Lineage"] = [str(val).split(".")[0] for val in lineages["Lineage_1"].values]
lineages.loc[lineages["Lineage"].str.contains("BOV"), "Lineage"] = "M. bovis"

assert len(lineages.loc[pd.isnull(lineages["Lineage"])]) == 0

########## KEEP ONLY ISOLATES WITH ALL 3 PIECES OF DATA ##########

# get only isolates with data for everyting: SNP matrix, in the model, and lineages
combined = lineages.merge(df_pza, left_on="Sample ID", right_on="sample_id")

In [42]:
combined.groupby("Lineage")["phenotype"].mean().sort_values(ascending=False)

Lineage
M. bovis   0.9153
2          0.3750
1          0.1503
6          0.1163
4          0.1112
5          0.1111
3          0.0554
7          0.0000
Name: phenotype, dtype: float64

In [45]:
combined.query("Lineage == 'M. bovis'").Lineage_1.unique()

array(['BOV_AFRI', 'BOV'], dtype=object)